To download data on google colab, upload kaggle.json and run cell below

In [1]:
# !mkdir -p /root/.kaggle
# !mv kaggle.json /root/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download -d franciscofeng/augmented-china-stock-data-with-fundamentals
# !unzip augmented-china-stock-data-with-fundamentals.zip -d data/

## Imports

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm
from collections import deque
from typing import Tuple, Optional, List

## Reading data

In [15]:
china_stocks = pd.read_csv("data/stock_data.csv")
ticker_info = pd.read_csv("data/ticker_info.csv")

In [16]:
china_stocks.head()

,date,ticker,open,high,low,close,volume,outstanding_share,turnover,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_mv,qfq_factor
0,2005-01-04,sh600000,0.77,0.77,0.75,0.76,3808939.0,900000000.0,0.004232,17.1990,14.4219,2.0777,3.1439,2.2097,6.9549,6.9549,2693520.0,8.895254
1,2005-01-05,sh600000,0.76,0.76,0.74,0.75,5225244.0,900000000.0,0.005806,16.9240,14.1913,2.0445,3.0937,2.1744,7.0679,7.0679,2650455.0,8.895254
2,2005-01-06,sh600000,0.75,0.75,0.73,0.74,4298099.0,900000000.0,0.004776,16.6991,14.0026,2.0173,3.0525,2.1455,7.1632,7.1632,2615220.0,8.895254
3,2005-01-07,sh600000,0.74,0.75,0.73,0.74,4362864.0,900000000.0,0.004848,16.7491,14.0446,2.0233,3.0617,2.1519,7.1418,7.1418,2623050.0,8.895254
4,2005-01-10,sh600000,0.75,0.77,0.74,0.77,7115260.0,900000000.0,0.007906,17.3240,14.0575,2.0082,3.1668,2.2258,6.9048,6.9048,2713095.0,8.895254


In [17]:
ticker_info.head()

,Unnamed: 0,ticker,company_name
0,0,sh600000,浦发银行
1,1,sh600004,白云机场
2,2,sh600006,东风汽车
3,3,sh600007,中国国贸
4,4,sh600008,首创环保


In [18]:
china_stocks['date'] = pd.to_datetime(china_stocks['date'])

In [19]:
china_stocks = china_stocks.pivot(index='date', columns='ticker', values='open')

In [20]:
train_china_stocks = china_stocks.loc[dt.datetime(2005, 1, 1) : dt.datetime(2020, 12, 31)]
val_china_stocks = china_stocks.loc[dt.datetime(2021, 1, 1) : dt.datetime(2021, 12, 31)]
test_china_stocks = china_stocks.loc[dt.datetime(2022, 1, 1) : dt.datetime(2022, 5, 11)]

In [9]:
del china_stocks

## V1

In [21]:
def drop_nans(a: np.ndarray, b: np.ndarray):
    inds = np.logical_not(np.isnan(a) | np.isnan(b))
    return a[inds], b[inds]


def correlation(a, b):
    return np.corrcoef(*drop_nans(a, b))[0, 1]


In [22]:
def zscore(a: np.ndarray):
    return (a - a.mean()) / a.std()


def ratio(a: np.ndarray, b: np.ndarray):
    a, b = drop_nans(a, b)
    return a / b


def zratio(a: np.ndarray, b: np.ndarray):
    return zscore(ratio(a, b))


In [61]:
class PairsTradingModel:
    def fit(
        self, 
        train: pd.DataFrame,
        correlation_threshold: float = 0.8, 
        nans_threshold: float = 0.2,
        ):
        """
        Finds most correlated pairs in given stock dataset
        """
        nans_threshold = int(len(train) * nans_threshold)
        train = train.loc[:, train.isnull().sum(axis=0) < nans_threshold]
        self.columns = train.columns
        train = train.to_numpy().T
        self.pairs = []
        for i in tqdm(range(train.shape[0]), "Finding correlated pairs"):
            for j in range(i + 1, train.shape[0]):
                corr = correlation(train[i], train[j])
                if corr > correlation_threshold:
                    self.pairs.append((i, j, corr))

        self.pairs.sort(key=lambda x: x[2], reverse=True)
    

    def test(
        self, 
        val: pd.DataFrame,
        pairs: List[Tuple[int, int, float]],
        desired_total_investement_per_pair: float = 100.,
        open_thresholds: Tuple[float, float] = (-1, 1),
        close_thresholds: Tuple[int, int] = (-0.4, 0.4),
        min_required_data_size: int = 30,
        ) -> Tuple[float, float]:

        """
        Tests strategy on data and returns tuple (sharpe ratio, return over max drawdown)
        """
        self._init_thresholds(open_thresholds, close_thresholds)

        n_pairs = len(pairs)
        val = val.loc[:, self.columns].to_numpy().T
        val = [(val[i], val[j], corr) for i, j, corr in pairs]
        
        max_days = max(max(len(i[0]), len(i[1])) for i in val)
        balances = np.asarray(
            [self.trade_pair(
                val[i][0], 
                val[i][1], 
                max_days, 
                desired_total_investement_per_pair,
                min_required_data_size=min_required_data_size,
                ) for i in tqdm(range(n_pairs), "testing")]
        ).sum(axis=0)
        
        returns = np.diff(balances) / balances[1:]

        # pairs trading is market neutral, so sharpe ratio doesn't need subtraction in formula
        sharpe_ratio = returns.mean() / returns.std()
        
        values = np.cumprod((returns + 1.))
        mdd = np.ptp(values) / values.max()
        romadd = returns[-1] / mdd

        return sharpe_ratio, romadd


    def trade_pair(
        self, 
        a: np.ndarray, 
        b: np.ndarray, 
        n_days: int, 
        desired_total_investement: float = 100.,
        min_required_data_size: int = 30,
        return_signals: bool = False
        ) -> np.ndarray:
        """
        Runs single pair backtest, returns array of portfolio balances each day
        """
        balances = []
        balance = 10000. # just for right percentage calculation
        n_a = 0
        n_b = 0

        signal_a = []
        signal_b = []

        rs = []
        for i in range(n_days):
            if i < len(a) and i < len(b) and\
                np.isfinite(a[i]) and np.isfinite(b[i]):
                rs.append(a[i] / b[i])
                if len(rs) >= min_required_data_size:
                    r = np.asarray(rs)
                    z = ((r - r.mean()) / r.std())[-1]
                    # make order if signal
                    if z < self.open_low or z > self.open_high:
                        cur_n_a, cur_n_b = self.calc_nums_a_b(
                            p_a=a[i], 
                            p_b=b[i], 
                            desired_total_investement=desired_total_investement, 
                            is_buying_a=(z < self.open_low))
                        balance -= (a[i] * cur_n_a + b[i] * cur_n_b)
                        signal_a.append(n_a)
                        signal_b.append(n_b)
                        n_a += cur_n_a
                        n_b += cur_n_b
                    elif self.close_low < z < self.close_high:
                        # close orders if signal
                        balance += (a[i] * n_a + b[i] * n_b)
                        signal_a.append(-n_a)
                        signal_b.append(-n_b)
                        n_a = 0
                        n_b = 0

                balances.append(balance + (a[i] * n_a + b[i] * n_b))
            else:
                balances.append(balance)

            if len(signal_a) == i:
                signal_a.append(0)
                signal_b.append(0)
        
        balances = np.asarray(balances)
        if return_signals:
            return np.asarray(signal_a), np.asarray(signal_b), balances
        else:
            return balances


    def calc_nums_a_b(
        self, 
        p_a: float, 
        p_b: float,
        desired_total_investement: float,
        is_buying_a: bool
        ):
        """
        This function finds quantity of stocks for order
        """
        n_a = desired_total_investement / (2 * p_a)
        n_b = desired_total_investement / (2 * p_b)
                    
        if is_buying_a:
            # selling b
            n_b = -n_b
        else:
            # selling a
            n_a = -n_a

        return n_a, n_b


    def plot_trading_pair(
        self, 
        df: pd.DataFrame,
        desired_total_investement: float = 100.,
        open_thresholds: Tuple[float, float] = (-1, 1),
        close_thresholds: Tuple[int, int] = (-0.2, 0.2),
        min_required_data_size: int = 30,
        ):
        """
        Plotting trading process for single pair
        """
        self._init_thresholds(open_thresholds, close_thresholds)
        name_a, name_b = df.columns
        a, b = df[name_a].to_numpy(), df[name_b].to_numpy()
        sig_a, sig_b, res_b = self.trade_pair(
            a=a, 
            b=b, 
            n_days=len(df), 
            desired_total_investement=desired_total_investement,
            min_required_data_size=min_required_data_size,
            return_signals=True
        )

        markers_size = 8

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df.index, 
            y=a,
            name=name_a,
            line_color='blue',
            mode='lines'
        ))

        fig.add_trace(go.Scatter(
            x=df.index,
            y=b,
            name=name_b,
            line_color='orange',
            mode='lines'
        ))

        # sell a
        fig.add_trace(go.Scatter(
            x=df.index[sig_a < 0],
            y=a[sig_a < 0],
            marker_symbol='arrow-down',
            marker_color='red',
            marker_size=markers_size,
            mode='markers',
            name=f'sell {name_a}'
        ))
        # sell b
        fig.add_trace(go.Scatter(
            x=df.index[sig_b < 0],
            y=b[sig_b < 0],
            marker_symbol='arrow-down',
            marker_color='red',
            marker_size=markers_size,
            mode='markers',
            name=f'sell {name_b}'
        ))
        # buy a
        fig.add_trace(go.Scatter(
            x=df.index[sig_a > 0],
            y=a[sig_a > 0],
            marker_symbol='arrow-up',
            marker_color='green',
            marker_size=markers_size,
            mode='markers',
            name=f'buy {name_a}'
        ))
        # buy b
        fig.add_trace(go.Scatter(
            x=df.index[sig_b > 0],
            y=b[sig_b > 0],
            marker_symbol='arrow-up',
            marker_color='green',
            marker_size=markers_size,
            mode='markers',
            name=f'buy {name_b}'
        ))
        fig.show()


    def _init_thresholds(self, open_thresholds, close_thresholds) -> None:
        self.open_low, self.open_high = open_thresholds
        self.close_low, self.close_high = close_thresholds


In [62]:
model = PairsTradingModel()

In [25]:
model.fit(train_china_stocks)

Finding correlated pairs:   0%|          | 0/1354 [00:00<?, ?it/s]

In [129]:
sharpe_ratio, romadd = model.test(test_china_stocks, pairs=model.pairs[:500], open_thresholds=(-1.1, 1.1), close_thresholds=(-1.09, 1.09))
print(f'Sharpe ratio: {sharpe_ratio}')
print(f'Return over max drawdown: {romadd}')

testing:   0%|          | 0/500 [00:00<?, ?it/s]

Sharpe ratio: 0.067750980755624
Return over max drawdown: 0.16296609931144074


In [130]:
tr_pair = val_china_stocks.loc[:, [model.columns[model.pairs[0][0]], model.columns[model.pairs[0][1]]]]
model.plot_trading_pair(tr_pair, open_thresholds=(-1.1, 1.1), close_thresholds=(-1.09, 1.09))

## V2: using cointegration instead of correlation

In [118]:
from statsmodels.tsa.stattools import coint

In [119]:
def cointegrated(a, b):
    a, b = drop_nans(a, b)
    coint_t, pvalue, _ = coint(a, b)
    return pvalue

In [120]:
class CointPairsTradingModel(PairsTradingModel):
    def fit(
        self, 
        train: pd.DataFrame,
        top_n: int = 25,
        correlation_threshold: float = 0.8, 
        nans_threshold: float = 0.2,
        ):
        """
        Finds most cointegrated pairs in given stock dataset
        """
        super().fit(train)
        train = train.loc[:, self.columns].to_numpy().T
        pairs = []
        for i, j, corr in tqdm(self.pairs[:1000], "Finding cointegrated pairs"):
            pvalue = cointegrated(train[i], train[j])
            if pvalue < 0.01:
                pairs.append((i, j, pvalue))

        self.pairs = pairs


In [121]:
coint_model = CointPairsTradingModel()

In [122]:
coint_model.fit(train_china_stocks)

Finding correlated pairs:   0%|          | 0/1354 [00:00<?, ?it/s]

Finding cointegrated pairs:   0%|          | 0/1000 [00:00<?, ?it/s]

In [132]:
sharpe_ratio, romadd = coint_model.test(test_china_stocks, pairs=coint_model.pairs[:500], open_thresholds=(-1.1, 1.1), close_thresholds=(-1.09, 1.09))
print(f'Sharpe ratio: {sharpe_ratio}')
print(f'Return over max drawdown: {romadd}')

testing:   0%|          | 0/500 [00:00<?, ?it/s]

Sharpe ratio: 0.07708378666443493
Return over max drawdown: 0.11796023551798394


In [145]:
tr_pair = val_china_stocks.loc[:, [coint_model.columns[coint_model.pairs[28][0]], coint_model.columns[coint_model.pairs[28][1]]]]
coint_model.plot_trading_pair(tr_pair, open_thresholds=(-1.1, 1.1), close_thresholds=(-1.09, 1.09))